In [73]:
#####################################################################################################
#
# Processamento dos arquivos JSON, gerados pela ingestao (pelo NIFI) da API Posicao da SPTRANS, 
# para a geração dos arquivos da camada Prata
# 


In [74]:
from pyspark.sql import SparkSession

In [75]:
from pyspark.sql.functions import explode

In [76]:
from pyspark.sql.functions import from_utc_timestamp, split, substring, lit, col

In [77]:
from datetime import datetime, timedelta
import zoneinfo as zi

In [78]:
spark = SparkSession.builder.appName("FIA-Proj-SPTRANS").enableHiveSupport().getOrCreate()

In [79]:
#####################################################################################################
#
# Leitura dos arquivos JSON ingeridos pelo NIFI da API Posicao, referentes a hora anterior a atual
#

In [80]:
#Calcula da hora anterior
GMT = zi.ZoneInfo('GMT')
LOCAL_TZ_STR='America/Sao_Paulo'
LOCAL_TZ = zi.ZoneInfo(LOCAL_TZ_STR)

dt_localtime=datetime.now(tz=LOCAL_TZ)
dt_lasthour= dt_localtime - timedelta(hours=0)

str_lasthour= dt_lasthour.strftime('%Y/%m/%d/%H')

str_data= dt_lasthour.strftime('%Y%m%d')


In [81]:
#Seta o path da camada bronze onde os arquivos da hora anterior serão lidos
bronze='s3a://bronze/API_SPTRANS_POSICAO_OK/' + str_lasthour + "/"

In [82]:
#Seta o path da camada prata onde serão persistidos os arquivos da camada prata
prata= 's3a://prata/POSICAO_PARQUET/' +  str_lasthour + "/"

In [83]:
print(bronze)
print(prata)

s3a://bronze/API_SPTRANS_POSICAO_OK/2024/09/18/20/
s3a://prata/POSICAO_PARQUET/2024/09/18/20/


In [84]:
#Lê os arquivos JSON retornados pela API Posicao
df_bronze= spark.read.json(bronze)

In [85]:
#Exibe o schema 
df_bronze.printSchema()

root
 |-- hr: string (nullable = true)
 |-- l: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- c: string (nullable = true)
 |    |    |-- cl: long (nullable = true)
 |    |    |-- lt0: string (nullable = true)
 |    |    |-- lt1: string (nullable = true)
 |    |    |-- qv: long (nullable = true)
 |    |    |-- sl: long (nullable = true)
 |    |    |-- vs: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- a: boolean (nullable = true)
 |    |    |    |    |-- is: string (nullable = true)
 |    |    |    |    |-- p: long (nullable = true)
 |    |    |    |    |-- px: double (nullable = true)
 |    |    |    |    |-- py: double (nullable = true)
 |    |    |    |    |-- sv: string (nullable = true)
 |    |    |    |    |-- ta: string (nullable = true)



In [86]:
#####################################################################################################
#
# Selecão das variáveis de interesse do JSON, normalizando as suas informacões e formatando-as em
# tabela
#

In [87]:
# Explodindo o array 'l' para criar uma nova linha para cada elemento do array
df_exploded = df_bronze.withColumn("l_exploded", explode("l"))

In [88]:
# Explodir o array 'vs' dentro do struct 'l_exploded'
df_vs_exploded = df_exploded.withColumn("vs_exploded", explode("l_exploded.vs"))

In [89]:
# Visualizar os campos de interesse
df_vs_exploded.select(
    col("hr").alias('hora_ref'),                  # hr : hora de referência da geração das infos
    col("vs_exploded.p").alias('cod_onibus'),     # p  : código do veículo    
    col("l_exploded.c").alias('let_cod_linha'),   # c  : código da linha no letreiro do ônibus    
    col("l_exploded.sl").alias('sentido_linha'),  # sl : sentido de operação da linha (1 do Term Principal para o Term Secundário - 2 do Term Secundário para o Term Principal)    
    col("l_exploded.lt0").alias('let_destino'),   # lt0: letreiro de destino da linha
    col("l_exploded.lt1").alias('let_origem'),    # lt1: letreiro de origem da linha
    from_utc_timestamp(split("vs_exploded.ta",'\+')[0],LOCAL_TZ_STR).alias('timestamp_pos'), #hora local da coleta das infos do ônibus
    col("vs_exploded.px").alias('latitude_pos'),  # px : latitude da posição do ônibus
    col("vs_exploded.py").alias('longitude_pos'), # py : longitude da posição do ônibus
    col("l_exploded.cl").alias('id_linha'),       # cl : código interno da linha
    col("l_exploded.qv").alias('qtde_onibus')     # qv : quantidade de ônibus localizados    
).show(5,truncate=False)

+--------+----------+-------------+-------------+-----------+----------+-------------------+-------------------+-------------------+--------+-----------+
|hora_ref|cod_onibus|let_cod_linha|sentido_linha|let_destino|let_origem|timestamp_pos      |latitude_pos       |longitude_pos      |id_linha|qtde_onibus|
+--------+----------+-------------+-------------+-----------+----------+-------------------+-------------------+-------------------+--------+-----------+
|20:47   |68116     |5010-10      |1            |STO. AMARO |JABAQUARA |2024-09-18 20:47:18|-46.7041145        |-23.6537065        |1114    |7          |
|20:47   |68434     |5010-10      |1            |STO. AMARO |JABAQUARA |2024-09-18 20:47:05|-46.631309         |-23.6624725        |1114    |7          |
|20:47   |68033     |5010-10      |1            |STO. AMARO |JABAQUARA |2024-09-18 20:46:44|-46.63600375       |-23.66687625       |1114    |7          |
|20:47   |68449     |5010-10      |1            |STO. AMARO |JABAQUARA |2024

In [90]:
# Criar dataframe com os campos de interesse, normalizando as infos necessárias, para gravação na camada prata
df_norm= df_vs_exploded.select(
    col("hr").alias('hora_ref'),                  # hr : hora de referência da geração das infos
    col("vs_exploded.p").alias('cod_onibus'),     # p  : código do veículo    
    col("l_exploded.sl").alias('sentido_linha'),  # sl : sentido de operação da linha (1 do Term Principal para o Term Secundário - 2 do Term Secundário para o Term Principal)        
    col("l_exploded.c").alias('let_cod_linha'),   # c  : código da linha no letreiro do ônibus    
    col("l_exploded.lt0").alias('let_destino'),   # lt0: letreiro de destino da linha
    col("l_exploded.lt1").alias('let_origem'),    # lt1: letreiro de origem da linha
    from_utc_timestamp(split("vs_exploded.ta",'\+')[0],LOCAL_TZ_STR).alias('timestamp_pos'), #hora local da coleta das infos do ônibus
    col("vs_exploded.px").alias('latitude_pos'),  # px : latitude da posição do ônibus
    col("vs_exploded.py").alias('longitude_pos'), # py : longitude da posição do ônibus
    col("l_exploded.cl").alias('id_linha'),       # cl : código interno da linha
    col("l_exploded.qv").alias('qtde_onibus')     # qv : quantidade de ônibus localizados    
)

In [91]:
df_norm_data= df_norm.withColumn("data_ref", lit(str_data) )

In [92]:
df_result= df_norm_data.select(
    "data_ref",       # data : data de referencia da geracao das infos
    'hora_ref',       # hr : hora de referência da geração das infos
    'cod_onibus',     # p  : código do veículo    
    'sentido_linha',  # sl : sentido de operação da linha (1 do Term Principal para o Term Secundário - 2 do Term Secundário para o Term Principal)        
    'let_cod_linha',  # c  : código da linha no letreiro do ônibus    
    'let_destino',    # lt0: letreiro de destino da linha
    'let_origem',     # lt1: letreiro de origem da linha
    'timestamp_pos',  #hora local da coleta das infos do ônibus
    'latitude_pos',   # px : latitude da posição do ônibus
    'longitude_pos',  # py : longitude da posição do ônibus
    'id_linha',       # cl : código interno da linha
    'qtde_onibus'     # qv : quantidade de ônibus localizados  
    )
                               

In [93]:
df_result.show(5)

+--------+--------+----------+-------------+-------------+-----------+----------+-------------------+-------------------+-------------------+--------+-----------+
|data_ref|hora_ref|cod_onibus|sentido_linha|let_cod_linha|let_destino|let_origem|      timestamp_pos|       latitude_pos|      longitude_pos|id_linha|qtde_onibus|
+--------+--------+----------+-------------+-------------+-----------+----------+-------------------+-------------------+-------------------+--------+-----------+
|20240918|   20:47|     68116|            1|      5010-10| STO. AMARO| JABAQUARA|2024-09-18 20:47:18|        -46.7041145|        -23.6537065|    1114|          7|
|20240918|   20:47|     68434|            1|      5010-10| STO. AMARO| JABAQUARA|2024-09-18 20:47:05|         -46.631309|        -23.6624725|    1114|          7|
|20240918|   20:47|     68033|            1|      5010-10| STO. AMARO| JABAQUARA|2024-09-18 20:46:44|       -46.63600375|       -23.66687625|    1114|          7|
|20240918|   20:47|   

In [94]:
#####################################################################################################
#
# Gravação do dataframe resultado do processamento na camada PRATA num path referente a última hora,
# em formato PARQUET.
# Obs: O processamento é feito para todos os arquivos da última hora, por isso utiliza-se o método
# overwrite para sobrepor os dados anteriores.
#

In [95]:
df_result.write.parquet(prata, mode='overwrite')

In [96]:
#####################################################################################################
#
# Fim do processamento